## **Helper Functions**

In [1]:
OPERATORS = ["&", "|", "!", "=", ">", "(", ")"]

In [2]:
# Function to check whether op is a propositional operator or not
def is_prop_operator(op):
    return op in OPERATORS

In [3]:
# Function parses the input sentence, identifies operators, skips the spaces and extracts literals
# Returns the list containing the operators and literals
def segment_sentence(sentence):
    i = 0
    L = len(sentence)
    segmented_sentence = []

    while i < L:
        if is_prop_operator(sentence[i]):
            segmented_sentence.append(sentence[i])
            i += 1
        elif sentence[i] == " ":
            i += 1
        else:
            literal = ""
            while i < L and not is_prop_operator(sentence[i]) and sentence[i] != " ":
                literal += sentence[i]
                i += 1
            segmented_sentence.append(literal)

    return segmented_sentence

In [4]:
# Function used to return the forward slice of sentence from index
def forward_slice(sentence, index):
    balance = 0
    i = index

    while i < len(sentence):
        if sentence[i] == "(":
          balance += 1
        else:
          if sentence[i] == ")":
            balance -= 1
          else:
            balance += 0
        if balance == 0 and sentence[i] != "!":
            return sentence[index : (i + 1)], i
        i += 1

In [6]:
# Function used to return the backward slice of sentence
def backward_slice(sentence):
    balance = 0
    L = len(sentence)
    i = L - 1

    while i >= 0:
        if sentence[i] == "(":
          balance += 1
        else:
          if sentence[i] == ")":
            balance -= 1
          else:
            balance += 0
        if balance == 0:
            i -= 1 if i > 0 and sentence[i - 1] == "!" else 0
            return sentence[i:L], sentence[0:i]
        i -= 1

In [10]:
def around_unary_op(sentence, op):
    processed_sentence = []
    i = 0
    while i < len(sentence):
        if sentence[i] == op:
            i += 1
            sentence_slice, i = forward_slice(sentence, i)
            sentence_slice = around_unary_op(sentence_slice, op)
            processed_sentence += ["(", "!"] + sentence_slice.copy() + [")"]
        else:
            processed_sentence.append(sentence[i])
        i += 1
    return processed_sentence

In [11]:
def around_binary_op(sentence, op):
    processed_sentence = []
    i = 0
    while i < len(sentence):
        if sentence[i] == op:
            A, processed_sentence = backward_slice(processed_sentence)
            A = around_binary_op(A, op)
            i += 1
            sentence_slice, i = forward_slice(sentence, i)
            sentence_slice = around_binary_op(sentence_slice, op)
            processed_sentence += (
                ["("] + A.copy() + [op] + sentence_slice.copy() + [")"]
            )
        else:
            processed_sentence.append(sentence[i])
        i += 1
    return processed_sentence

In [12]:
def induce_parenthesis(sentence):
    sentence = around_unary_op(sentence, "!")
    sentence = around_binary_op(sentence, "&")
    sentence = around_binary_op(sentence, "|")
    sentence = around_binary_op(sentence, ">")
    sentence = around_binary_op(sentence, "=")
    return sentence

In [13]:
def literal_not_protected(sentence):
    if not any(is_prop_operator(i) for i in sentence):
        return False

    off_balance = 0
    for i in range(0, len(sentence)):
        if sentence[i] == "(":
            off_balance += 1
        elif sentence[i] == ")":
            off_balance -= 1
        elif off_balance == 0:
            return True

    return False

In [31]:
# Function returns ((A>B)&(B>A)) as equivalent of (A=B)
def iff_equivalent(A, B):
    return (
        ["(", "("]
        + A.copy()
        + [">"]
        + B.copy()
        + [")", "&", "("]
        + B.copy()
        + [">"]
        + A.copy()
        + [")", ")"]
    )

In [8]:
# Function returns ((!A)|B) as equivalent of (A>B)
def implies_equivalent(A, B):
    return ["(", "(", "!"] + A.copy() + [")", "|"] + B.copy() + [")"]

In [32]:
# Function identifies instances of the operator and replacing them with their equivalent expressions
def eliminate_op(sentence, op):
    processed_sentence = []
    i = 0
    while i < len(sentence):
        if sentence[i] == op:
            A, processed_sentence = backward_slice(processed_sentence)
            i += 1
            B, i = forward_slice(sentence, i)
            A = eliminate_op(A, op)
            B = eliminate_op(B, op)
            processed_sentence += (
                iff_equivalent(A, B) if op == "=" else implies_equivalent(A, B)
            ).copy()
        else:
            processed_sentence.append(sentence[i])
        i += 1
    return processed_sentence

In [15]:
def process_operand(operand):
    operand = eliminate_invalid_parenthesis(operand)
    if operand[0] == "(":
        return (
            ["("]
            + [j for j in operand[1 : len(operand) - 1] if j not in ["(", ")"]]
            + [")"]
        )
    else:
        return [j for j in operand[0 : len(operand)] if j not in ["(", ")"]]



In [16]:
def split_around_and(sentence):
    processed_sentence = []
    operand = []

    for i in range(0, len(sentence)):
        if sentence[i] == "&":
            processed_sentence += process_operand(operand).copy()
            processed_sentence.append("&")
            operand.clear()
        else:
            operand.append(sentence[i])

    return processed_sentence + process_operand(operand).copy()

In [17]:
def move_not_inwards(sentence):
    processed_sentence = []

    while True:
        processed_sentence = []
        i = 0
        while i < len(sentence):
            if sentence[i] == "!":
                i += 1
                B, i = forward_slice(sentence, i)
                if B[0] == "(":
                    processed_sentence.append("(")
                    j = 1
                    while j < len(B):
                        tmp, j = forward_slice(B, j)
                        tmp.pop(0) if tmp[0] == "!" else tmp.insert(0, "!")
                        processed_sentence += tmp.copy()
                        j += 1
                        if j < len(B) - 1:
                            processed_sentence += (
                                ["&"] if B[j] == "|" else ["|"] if (B[j] == "&") else []
                            )
                        j += 1
                    processed_sentence.append(")")
                else:
                    B.pop(0) if B[0] == "!" else processed_sentence.append("!")
                    processed_sentence += B.copy()
            else:
                processed_sentence.append(sentence[i])
            i += 1

        if processed_sentence == sentence:
            break
        sentence = processed_sentence

    return processed_sentence

In [18]:
def distribute_or_over_and(sentence):
    processed_sentence = []

    i = 0
    while i < len(sentence):
        if sentence[i] == "|":
            A, processed_sentence = backward_slice(processed_sentence)
            A = distribute_or_over_and(A)

            tmp3 = []
            if A[0] == "(":
                j = 1
                while j < (len(A) - 1):
                    tmp, j = forward_slice(A, j)
                    tmp3.append(tmp.copy())
                    j += 2
            else:
                tmp3.append(A.copy())

            i += 1
            assert i < len(sentence)

            B, i = forward_slice(sentence, i)
            B = distribute_or_over_and(B)

            tmp2 = []
            if B[0] == "(":
                j = 1
                while j < (len(B) - 1):
                    tmp, j = forward_slice(B, j)
                    tmp2.append(tmp.copy())
                    j += 2
            else:
                tmp2.append(B.copy())

            for k in range(0, len(tmp2)):
                for m in range(0, len(tmp3)):
                    processed_sentence += (
                        ["("]
                        + tmp3[m].copy()
                        + ["|"]
                        + tmp2[k].copy()
                        + [")"]
                        + (["&"] if m != len(tmp3) - 1 else [])
                    )
                processed_sentence += ["&"] if k != len(tmp2) - 1 else []
        else:
            processed_sentence.append(sentence[i])

        i += 1

    return processed_sentence

In [19]:
def eliminate_invalid_parenthesis(sentence):
    processed_sentence = []
    brackets = []
    content = []

    for i in range(0, len(sentence)):
        if sentence[i] == "(":
            content.append(processed_sentence.copy())
            brackets.append("(")
            processed_sentence.clear()
        elif sentence[i] == ")" and len(content):
            if literal_not_protected(processed_sentence):
                processed_sentence = ["("] + processed_sentence + [")"]
            processed_sentence = content[len(content) - 1].copy() + processed_sentence
            brackets.pop()
            content.pop()
        else:
            processed_sentence.append(sentence[i])

    return processed_sentence

### **Part(0) - Function to convert each formula into CNF**

In [25]:
# Function returns Conjunctive Normal Form (CNF) of the sentence.
def CNF(sentence):
    sentence = eliminate_op(sentence, "=")
    sentence = eliminate_invalid_parenthesis(sentence)
    sentence = eliminate_op(sentence, ">")
    sentence = eliminate_invalid_parenthesis(sentence)
    sentence = move_not_inwards(sentence)
    sentence = eliminate_invalid_parenthesis(sentence)

    prev = []
    while prev != sentence:
        prev = sentence
        sentence = distribute_or_over_and(sentence)
        sentence = eliminate_invalid_parenthesis(sentence)

    return split_around_and(sentence)

In [26]:
# Function returns the sentence in CNF after vetting sentence with proper parenthesis
def vet_sentence(sentence):
    sentence = induce_parenthesis(sentence)
    sentence = eliminate_invalid_parenthesis(sentence)
    return CNF(sentence)

In [20]:
# Function returns a map where keys are literals in sentence and value is True if they are negated and False otherwise.
def clause_map(sentence):
    m = {}
    j = 1 if sentence[0] == "(" else 0
    L = len(sentence) - 1 if sentence[0] == "(" else len(sentence)
    while j < L:
        literal, j = forward_slice(sentence, j)
        if literal[0] == "!":
            m[literal[1]] = True
        else:
            m[literal[0]] = False
        j += 2
    return m

In [21]:
def format_dict(dictionary):
    return ", ".join(
        [("!" if dictionary[key] else "") + str(key) for key in dictionary]
    )

In [9]:
# Function to take the input in the desired format as mentioned in assignment
# Returns m, list of propositional sentences and query

def _get_input():
    n, m = input().split()
    n = int(n)
    m = int(m)

    sentences = []
    while n:
        sentences.append(input().splitlines()[0])
        n -= 1

    query = input().splitlines()[0]

    return m, sentences, query

### **Implement the resolution-refutation algorithm posing it as an uninformed search problem**

In [101]:
# Function to resolve the given sentence and to print or not to print the resolution steps
def resolve(sentence, mode):
    clause = []
    clauses = []
    clause_maps = []
    for literal in sentence:
        if literal == "&":
            clauses.append("".join(clause))
            clause_maps.append(clause_map(clause))
            clause.clear()
        else:
            clause.append(literal)
    clauses.append("".join(clause))
    clause_maps.append(clause_map(clause))
    new_clause_maps = []

    if mode:
        print("Clauses <- The set of clauses in the CNF representation of (KB & !Q)")
        print("Clauses: {}".format(clauses))
        print("New Clauses <- {}")
        print("For each pair of clauses C_i, C_j in Clauses do:")

    while True:
        for i in range(0, len(clause_maps)):
            for j in range((i + 1), len(clause_maps)):
                resolvent = {}
                for var in clause_maps[i]:
                    if (
                        var not in clause_maps[j]
                        or clause_maps[j][var] == clause_maps[i][var]
                    ):
                        resolvent[var] = clause_maps[i][var]

                for var in clause_maps[j]:
                    if var not in clause_maps[i]:
                        resolvent[var] = clause_maps[j][var]

                print(
                    "\t({}) <- RESOLVE(({}), ({}))".format(
                        format_dict(resolvent),
                        format_dict(clause_maps[i]),
                        format_dict(clause_maps[j]),
                    )
                ) if mode else None

                if not bool(resolvent):
                    print(
                        "\tIf Resolvents contains the empty clause: Return True."
                    ) if mode else None
                    return True

                new_clause_maps.append(
                    resolvent
                ) if resolvent not in new_clause_maps else None
                print("\tNew Clauses <- New Clauses ∪ Resolvents") if mode else None

        if all(new_clause_map in clause_maps for new_clause_map in new_clause_maps):
            print("If New Clauses ⊆ Clauses : Return False") if mode else None
            return False

        clause_maps += [
            new_clause_map
            for new_clause_map in new_clause_maps
            if new_clause_map not in clause_maps
        ]
        print("Clauses <- Clauses ∪ New Clauses") if mode else None

In [100]:
from queue import Queue

def generate_neighbors(sentence):
    neighbors = []
    for i in range(len(sentence)):
        if sentence[i] == "&":
            # Identify clauses to resolve
            clause_1, _ = backward_slice(sentence[:i])
            clause_2, _ = forward_slice(sentence, i + 1)

            resolvent = resolve_clauses(clause_1, clause_2)

            if resolvent is not None:
                # Creating a new state with the resolved clauses
                new_state = sentence[:i - len(clause_1)] + resolvent + sentence[i + len(clause_2) + 1:]
                neighbors.append(new_state)

    return neighbors

# Function to resolve two clauses and returns the resolvent if applicable.
def resolve_clauses(clause_1, clause_2):
    clause_map_1 = clause_map(clause_1)
    clause_map_2 = clause_map(clause_2)

    resolvent = {}
    for var in clause_map_1:
        if var in clause_map_2 and clause_map_2[var] != clause_map_1[var]:
            return None  # Contradiction, cannot resolve

        resolvent[var] = clause_map_1[var]

    for var in clause_map_2:
        if var not in clause_map_1:
            resolvent[var] = clause_map_2[var]

    return [format_dict(resolvent)]

In [102]:
def uninformed_search(knowledge_base, query, mode):
    q = Queue()
    initial_state = vet_sentence(knowledge_base + ["&", "(", "!", "(", *query, ")", ")"])
    q.put(initial_state)
    num_explored_nodes = 0

    while not q.empty():
        current_state= q.get()
        num_explored_nodes += 1

        if resolve(current_state, mode):
            return 1,num_explored_nodes  # Goal state reached (unsatisfiability)

        # Explore neighboring states (apply resolution steps)
        for neighbor in generate_neighbors(current_state):
            q.put(neighbor)

    return 0,num_explored_nodes  # No empty clause found


In [113]:
if __name__ == "__main__":
    m, sentences, query = _get_input()

    knowledge_base = []
    for sentence in sentences:
        knowledge_base += vet_sentence(segment_sentence(sentence)).copy()
        knowledge_base.append("&")
    knowledge_base.pop()

    query = vet_sentence(segment_sentence(query))

    result,num_explored_nodes = uninformed_search(knowledge_base, query, m)

    # If m == 0 :
    #   Printing only the result (integer 0 or 1)
    # If m == 1 :
    #   Printing the resolution steps
    #   Printing the result (integer 0 or 1) in the last line

    print("Result: ",result)
    print("Number of Nodes Explored:", num_explored_nodes)

6 0
A>B
!B
!A>(C|D)
C>E
F>!E
F
D
Result:  1
Number of Nodes Explored: 1


### **Implement the resolution-refutation algorithm posing it as a greedy search problem. For this, design a task-specific heuristic function**

In [109]:
# A heuristic function based on the number of literals in a clause.
def heuristic(clause):
    return len([literal for literal in clause if literal != "&"])

In [110]:
def resolve_greedy(sentence, mode):
    clause = []
    clauses = []
    clause_maps = []
    node_counter=0
    for literal in sentence:
        if literal == "&":
            clauses.append("".join(clause))
            clause_maps.append(clause_map(clause))
            clause.clear()
        else:
            clause.append(literal)
    clauses.append("".join(clause))
    clause_maps.append(clause_map(clause))
    new_clause_maps = []

    if mode:
        print("Clauses <- The set of clauses in the CNF representation of (KB & !Q)")
        print("Clauses: {}".format(clauses))
        print("New Clauses <- {}")
        print("For each pair of clauses C_i, C_j in Clauses do:")

    while True:
        for i in range(0, len(clause_maps)):
            for j in range((i + 1), len(clause_maps)):
                node_counter+=1
                resolvent = {}
                for var in clause_maps[i]:
                    if (
                        var not in clause_maps[j]
                        or clause_maps[j][var] == clause_maps[i][var]
                    ):
                        resolvent[var] = clause_maps[i][var]

                for var in clause_maps[j]:
                    if var not in clause_maps[i]:
                        resolvent[var] = clause_maps[j][var]

                print(
                    "\t({}) <- RESOLVE(({}), ({}))".format(
                        format_dict(resolvent),
                        format_dict(clause_maps[i]),
                        format_dict(clause_maps[j]),
                    )
                ) if mode else None

                if not bool(resolvent):
                    print(
                        "\tIf Resolvents contains the empty clause: Return True."
                    ) if mode else None
                    return 1, node_counter

                new_clause_maps.append(resolvent) if resolvent not in new_clause_maps else None
                print("\tNew Clauses <- New Clauses ∪ Resolvents") if mode else None

        if all(new_clause_map in clause_maps for new_clause_map in new_clause_maps):
            print("If New Clauses ⊆ Clauses : Return False") if mode else None
            return 0,node_counter

        # Sort new_clause_maps based on the heuristic
        new_clause_maps.sort(key=heuristic)

        clause_maps += [
            new_clause_map
            for new_clause_map in new_clause_maps
            if new_clause_map not in clause_maps
        ]
        print("Clauses <- Clauses ∪ New Clauses") if mode else None


In [112]:
def main():
    m, sentences, query = _get_input()

    knowledge_base = []
    for sentence in sentences:
        sentence = vet_sentence(segment_sentence(sentence))
        knowledge_base += sentence.copy()
        knowledge_base.append("&")
    knowledge_base.pop()

    query = vet_sentence(segment_sentence(query))

    # rub : Input for resolution
    rub = ["!"] + query.copy()
    if len(knowledge_base) > 0:
        rub = knowledge_base.copy() + ["&", "("] + rub.copy() + [")"]

    if len(rub):
        rub = vet_sentence(rub)
        result, node = resolve_greedy(rub, m)
        print("Result: ",result)
        print("Total Nodes Explored: ",node)

if __name__ == "__main__":
    main()

3 1
(P>Q)>Q
(P>P)>R
(R>S)>!(S>Q)
R
Clauses <- The set of clauses in the CNF representation of (KB & !Q)
Clauses: ['(P|Q)', '(!Q|Q)', '(P|R)', '(!P|R)', '(R|S)', '(!S|S)', '(R|!Q)', '(!S|!Q)', '(!R)']
New Clauses <- {}
For each pair of clauses C_i, C_j in Clauses do:
	(P, Q) <- RESOLVE((P, Q), (Q))
	New Clauses <- New Clauses ∪ Resolvents
	(P, Q, R) <- RESOLVE((P, Q), (P, R))
	New Clauses <- New Clauses ∪ Resolvents
	(Q, R) <- RESOLVE((P, Q), (!P, R))
	New Clauses <- New Clauses ∪ Resolvents
	(P, Q, R, S) <- RESOLVE((P, Q), (R, S))
	New Clauses <- New Clauses ∪ Resolvents
	(P, Q, S) <- RESOLVE((P, Q), (S))
	New Clauses <- New Clauses ∪ Resolvents
	(P, R) <- RESOLVE((P, Q), (R, !Q))
	New Clauses <- New Clauses ∪ Resolvents
	(P, !S) <- RESOLVE((P, Q), (!S, !Q))
	New Clauses <- New Clauses ∪ Resolvents
	(P, Q, !R) <- RESOLVE((P, Q), (!R))
	New Clauses <- New Clauses ∪ Resolvents
	(Q, P, R) <- RESOLVE((Q), (P, R))
	New Clauses <- New Clauses ∪ Resolvents
	(Q, !P, R) <- RESOLVE((Q), (!P, R))